In [4]:
from google.oauth2 import service_account
from google.cloud import bigquery
from google.cloud.exceptions import NotFound
import pandas as pd
from google.cloud import storage

In [5]:
project_id = "looker-assignment-113356033"
bucket_name = "bucket_midterm_113356033"
dataset_id = "midterm_dataset"

In [6]:
crendentials = service_account.Credentials.from_service_account_file(r"C:\Users\HungLin\Desktop\workspace\data_model_midterm\looker-assignment-113356033-f5936d88b9bd.json")

In [7]:
bigquery_client = bigquery.Client(project=project_id, credentials=crendentials)
storage_client = storage.Client(credentials=crendentials, project=project_id)

**Question 1**

建立Bucket

In [8]:
bucket = storage_client.bucket(bucket_name)
try:
    storage_client.get_bucket(bucket)
    print(f"Bucket '{bucket_name}' has already exists.")
except Exception:
    new_bucket = storage_client.create_bucket(bucket, location="US")
    new_bucket.storage_class = 'STANDARD'
    new_bucket.patch()
    print(f"Bucket '{bucket_name}' created successfully with storage class '{new_bucket.storage_class}' in location '{new_bucket.location}.")

Bucket 'bucket_midterm_113356033' has already exists.


將csv上傳到bucket

In [9]:
blob = bucket.blob('ad_performance.csv') #欲建立的csv檔名稱
blob.upload_from_filename('ad_performance.csv') #本地端要上傳的csv檔

In [10]:
dataset_ref = bigquery_client.dataset(dataset_id)
try:
    bigquery_client.get_dataset(dataset_ref)
except Exception:
    bigquery_client.create_dataset(dataset_id)

In [11]:
query_job = bigquery_client.query("""
    CREATE OR REPLACE EXTERNAL TABLE midterm_dataset.ad_performance
    OPTIONS(
        format = 'CSV',
        uris = ['gs://bucket_midterm_113356033/ad_performance.csv']
    );
""")

In [12]:
results = query_job.result()
df = results.to_dataframe()

In [13]:
blob = bucket.blob('customer_logs.csv') #欲建立的csv檔名稱
blob.upload_from_filename('customer_logs.csv') #本地端要上傳的csv檔

In [14]:
dataset_ref = bigquery_client.dataset(dataset_id)
try:
    bigquery_client.get_dataset(dataset_ref)
except Exception:
    bigquery_client.create_dataset(dataset_id)

In [15]:
query_job = bigquery_client.query("""
    CREATE OR REPLACE EXTERNAL TABLE midterm_dataset.customer_logs
    OPTIONS(
        format = 'CSV',
        uris = ['gs://bucket_midterm_113356033/customer_logs.csv']
    );
""")

In [16]:
results = query_job.result()
df = results.to_dataframe()

從google_trends.top_rising_terms提取資料

In [17]:
google_trends_query = """
WITH RankedData AS (
    SELECT
        term,
        score,
        country_code,
        country_name,
        ROW_NUMBER() OVER (PARTITION BY country_code) AS row_num
    FROM `bigquery-public-data.google_trends.international_top_rising_terms`
    WHERE score IS NOT NULL
)
SELECT country_code, country_name, term, score
FROM RankedData
WHERE MOD(row_num, 10000) = 0
"""

google_trends_data = bigquery_client.query(google_trends_query).to_dataframe() 

c:\Users\HungLin\anaconda3\Lib\site-packages\google\cloud\bigquery\table.py:1727: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


In [18]:
google_trends_data

,country_code,country_name,term,score
0,DK,Denmark,feriekonto,25
1,DK,Denmark,praktikportalen,34
2,DK,Denmark,danløn,79
3,DK,Denmark,superliga stilling,16
4,DK,Denmark,fck,26
...,...,...,...,...
3962,RO,Romania,bt24,70
3963,RO,Romania,loto 6/49,15
3964,RO,Romania,punct unic de acces,55
3965,RO,Romania,fcsb,11


進行regulization

In [19]:
from sklearn.preprocessing import MinMaxScaler

# 使用 MinMaxScaler 正規化 score
scaler = MinMaxScaler()
google_trends_data['score'] = scaler.fit_transform(google_trends_data[['score']])
print(google_trends_data.head())

  country_code country_name                term     score
0           DK      Denmark          feriekonto  0.242424
1           DK      Denmark     praktikportalen  0.333333
2           DK      Denmark              danløn  0.787879
3           DK      Denmark  superliga stilling  0.151515
4           DK      Denmark                 fck  0.252525


In [20]:
#提取其他要merge的資料 
geo_data = bigquery_client.query("""
SELECT criteria_id, en_name, country_code
FROM `bigquery-public-data.google_ads.geotargets`
""").to_dataframe()

geo_data['criteria_id'] = geo_data['criteria_id'].astype('Int64')

ads_geotargets = bigquery_client.query("""
SELECT ads_criteria_id, target_city
FROM `bigquery-public-data.google_ads_geo_mapping_us.ads_geo_criteria_mapping`
""").to_dataframe()

google_analytics_data = bigquery_client.query("""
SELECT geoNetwork.country, totals.pageviews, totals.transactions
FROM `bigquery-public-data.google_analytics_sample.ga_sessions_20170801`
""").to_dataframe()

ad_performance = bigquery_client.query("""
SELECT *
FROM `looker-assignment-113356033.midterm_dataset.ad_performance`
""").to_dataframe()

customer_logs = bigquery_client.query("""
SELECT *
FROM `looker-assignment-113356033.midterm_dataset.customer_logs`
""").to_dataframe()

c:\Users\HungLin\anaconda3\Lib\site-packages\google\cloud\bigquery\table.py:1727: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(
c:\Users\HungLin\anaconda3\Lib\site-packages\google\cloud\bigquery\table.py:1727: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(
c:\Users\HungLin\anaconda3\Lib\site-packages\google\cloud\bigquery\table.py:1727: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(
c:\Users\HungLin\anaconda3\Lib\site-packages\google\cloud\bigquery\table.py:1727: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(
c:\Users\HungLin\anaconda3\Lib\site-packages\google\cloud\bigquery\table.py:1727: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


In [21]:
print(geo_data)

        criteria_id                   en_name country_code
0           1000998                      Baku           AZ
1           9069514                  Sumqayit           AZ
2           1000694  Wimpassing an der Leitha           AT
3           9070053                   Yerevan           AM
4           1001784               Saint James           BB
...             ...                       ...          ...
105605      9040813                      WI-8           US
105606      9040814                      WV-1           US
105607      9040815                      WV-2           US
105608      9040816                      WV-3           US
105609      9040817               WY-AT LARGE           US

[105610 rows x 3 columns]


In [22]:
print(ads_geotargets)

       ads_criteria_id target_city
0              1009819    Temirtau
1                 2398        None
2              1009820   Turkistan
3              1009805      Almaty
4              1009814   Lisakovsk
...                ...         ...
59758          9057016        None
59759          9058077        None
59760          9057010        None
59761          9057021        None
59762             2666        None

[59763 rows x 2 columns]


In [23]:
print(google_analytics_data)

             country  pageviews  transactions
0             Greece          1          <NA>
1              India          1          <NA>
2     United Kingdom          1          <NA>
3      United States          1          <NA>
4      United States          1          <NA>
...              ...        ...           ...
2551   United States         60          <NA>
2552   United States         72          <NA>
2553   United States         93          <NA>
2554   United States        112             1
2555   United States        106             2

[2556 rows x 3 columns]


Match Ads Geotargets and Geo Mapping on criteria_id.

In [24]:

geo_ads_merged = pd.merge(ads_geotargets, geo_data, left_on="ads_criteria_id", right_on="criteria_id", how="inner")
geo_ads_merged


,ads_criteria_id,target_city,criteria_id,en_name,country_code
0,2398,None,2398,Kazakhstan,KZ
1,1009806,Astana,1009806,Nur-Sultan,KZ
2,9070289,None,9070289,Akmola Province,KZ
3,9063098,Aktobe,9063098,Aktobe,KZ
4,9075589,None,9075589,Aktobe Province,KZ
...,...,...,...,...,...
48251,9057016,None,9057016,Lake and Peninsula,US
48252,9058077,None,9058077,St. John the Baptist Parish,US
48253,9057010,None,9057010,Fairbanks North Star,US
48254,9057021,None,9057021,Prince of Wales-Hyder Census Area,US


In [25]:
geo_ads_merged = geo_ads_merged.drop(['target_city', 'criteria_id', 'en_name'], axis=1)
geo_ads_merged

,ads_criteria_id,country_code
0,2398,KZ
1,1009806,KZ
2,9070289,KZ
3,9063098,KZ
4,9075589,KZ
...,...,...
48251,9057016,US
48252,9058077,US
48253,9057010,US
48254,9057021,US


In [26]:
matched_criteria = google_trends_data["country_code"].isin(geo_ads_merged["country_code"]).sum()
print(f"Matched rows: {matched_criteria}")

Matched rows: 3967


In [27]:
# 匹配 Trends 數據與 Ads Geotargets
trends_ads_merged = pd.merge(google_trends_data, geo_ads_merged, on="country_code", how="inner")

trends_ads_merged

,country_code,country_name,term,score,ads_criteria_id
0,DK,Denmark,feriekonto,0.242424,2208
1,DK,Denmark,feriekonto,0.242424,9067616
2,DK,Denmark,feriekonto,0.242424,9067623
3,DK,Denmark,feriekonto,0.242424,9048951
4,DK,Denmark,feriekonto,0.242424,1005125
...,...,...,...,...,...
2651608,RO,Romania,radio zu,0.555556,1011812
2651609,RO,Romania,radio zu,0.555556,9051354
2651610,RO,Romania,radio zu,0.555556,1011799
2651611,RO,Romania,radio zu,0.555556,20892


In [28]:
# 匹配 Trends 數據與 Ads Geotargets
trends_ads_merged = trends_ads_merged.drop_duplicates()

trends_ads_merged

,country_code,country_name,term,score,ads_criteria_id
0,DK,Denmark,feriekonto,0.242424,2208
1,DK,Denmark,feriekonto,0.242424,9067616
2,DK,Denmark,feriekonto,0.242424,9067623
3,DK,Denmark,feriekonto,0.242424,9048951
4,DK,Denmark,feriekonto,0.242424,1005125
...,...,...,...,...,...
2651608,RO,Romania,radio zu,0.555556,1011812
2651609,RO,Romania,radio zu,0.555556,9051354
2651610,RO,Romania,radio zu,0.555556,1011799
2651611,RO,Romania,radio zu,0.555556,20892


In [29]:
ad_performance

,ad_id,region_id,impressions,clicks,conversions,country
0,AD001,R01,427,1294,724,Norway
1,AD002,R09,3466,1966,985,United Kingdom
2,AD003,R10,6163,2221,486,Spain
3,AD004,R06,1823,4996,974,Switzerland
4,AD005,R06,2007,2899,746,Malaysia
5,AD006,R04,9379,4534,949,Vietnam
6,AD007,R05,4306,3576,558,Indonesia
7,AD008,R03,3383,4955,239,Ukraine
8,AD009,R04,7886,4516,97,Turkey
9,AD010,R05,8722,1451,768,Canada


In [30]:
customer_logs

,log_id,timestamp,ad_id,user_action,Country
0,LOG036,2024-01-02 11:00:00+00:00,AD001,click,Norway
1,LOG041,2024-01-02 16:00:00+00:00,AD001,view,Norway
2,LOG016,2024-01-01 15:00:00+00:00,AD002,view,United Kingdom
3,LOG026,2024-01-02 01:00:00+00:00,AD002,view,United Kingdom
4,LOG048,2024-01-02 23:00:00+00:00,AD002,click,United Kingdom
5,LOG028,2024-01-02 03:00:00+00:00,AD003,purchase,Spain
6,LOG046,2024-01-02 21:00:00+00:00,AD003,click,Spain
7,LOG001,2024-01-01 00:00:00+00:00,AD004,click,Switzerland
8,LOG014,2024-01-01 13:00:00+00:00,AD004,purchase,Switzerland
9,LOG037,2024-01-02 12:00:00+00:00,AD004,view,Switzerland


In [31]:
customer_logs = customer_logs.drop(['ad_id', 'log_id'], axis=1)
customer_logs

,timestamp,user_action,Country
0,2024-01-02 11:00:00+00:00,click,Norway
1,2024-01-02 16:00:00+00:00,view,Norway
2,2024-01-01 15:00:00+00:00,view,United Kingdom
3,2024-01-02 01:00:00+00:00,view,United Kingdom
4,2024-01-02 23:00:00+00:00,click,United Kingdom
5,2024-01-02 03:00:00+00:00,purchase,Spain
6,2024-01-02 21:00:00+00:00,click,Spain
7,2024-01-01 00:00:00+00:00,click,Switzerland
8,2024-01-01 13:00:00+00:00,purchase,Switzerland
9,2024-01-02 12:00:00+00:00,view,Switzerland


In [32]:
performance_customer_merged = pd.merge(ad_performance, customer_logs, left_on="country", right_on="Country")
performance_customer_merged

,ad_id,region_id,impressions,clicks,conversions,country,timestamp,user_action,Country
0,AD001,R01,427,1294,724,Norway,2024-01-02 11:00:00+00:00,click,Norway
1,AD001,R01,427,1294,724,Norway,2024-01-02 16:00:00+00:00,view,Norway
2,AD002,R09,3466,1966,985,United Kingdom,2024-01-01 15:00:00+00:00,view,United Kingdom
3,AD002,R09,3466,1966,985,United Kingdom,2024-01-02 01:00:00+00:00,view,United Kingdom
4,AD002,R09,3466,1966,985,United Kingdom,2024-01-02 23:00:00+00:00,click,United Kingdom
5,AD003,R10,6163,2221,486,Spain,2024-01-02 03:00:00+00:00,purchase,Spain
6,AD003,R10,6163,2221,486,Spain,2024-01-02 21:00:00+00:00,click,Spain
7,AD004,R06,1823,4996,974,Switzerland,2024-01-01 00:00:00+00:00,click,Switzerland
8,AD004,R06,1823,4996,974,Switzerland,2024-01-01 13:00:00+00:00,purchase,Switzerland
9,AD004,R06,1823,4996,974,Switzerland,2024-01-02 12:00:00+00:00,view,Switzerland


In [33]:
performance_customer_merged = performance_customer_merged.drop(['region_id', 'Country'], axis=1)
performance_customer_merged

,ad_id,impressions,clicks,conversions,country,timestamp,user_action
0,AD001,427,1294,724,Norway,2024-01-02 11:00:00+00:00,click
1,AD001,427,1294,724,Norway,2024-01-02 16:00:00+00:00,view
2,AD002,3466,1966,985,United Kingdom,2024-01-01 15:00:00+00:00,view
3,AD002,3466,1966,985,United Kingdom,2024-01-02 01:00:00+00:00,view
4,AD002,3466,1966,985,United Kingdom,2024-01-02 23:00:00+00:00,click
5,AD003,6163,2221,486,Spain,2024-01-02 03:00:00+00:00,purchase
6,AD003,6163,2221,486,Spain,2024-01-02 21:00:00+00:00,click
7,AD004,1823,4996,974,Switzerland,2024-01-01 00:00:00+00:00,click
8,AD004,1823,4996,974,Switzerland,2024-01-01 13:00:00+00:00,purchase
9,AD004,1823,4996,974,Switzerland,2024-01-02 12:00:00+00:00,view


In [34]:
trends_ads_perf_cus_merged = pd.merge(trends_ads_merged, performance_customer_merged, left_on="country_name", right_on="country")
trends_ads_perf_cus_merged

,country_code,country_name,term,score,ads_criteria_id,ad_id,impressions,clicks,conversions,country,timestamp,user_action
0,CA,Canada,cyber monday deals,0.828283,2124,AD010,8722,1451,768,Canada,2024-01-01 09:00:00+00:00,view
1,CA,Canada,cyber monday deals,0.828283,2124,AD010,8722,1451,768,Canada,2024-01-01 18:00:00+00:00,view
2,CA,Canada,cyber monday deals,0.828283,2124,AD010,8722,1451,768,Canada,2024-01-02 06:00:00+00:00,click
3,CA,Canada,cyber monday deals,0.828283,1002806,AD010,8722,1451,768,Canada,2024-01-01 09:00:00+00:00,view
4,CA,Canada,cyber monday deals,0.828283,1002806,AD010,8722,1451,768,Canada,2024-01-01 18:00:00+00:00,view
...,...,...,...,...,...,...,...,...,...,...,...,...
4388388,NL,Netherlands,vitesse,0.161616,9063453,AD013,2641,137,495,Netherlands,2024-01-01 20:00:00+00:00,purchase
4388389,NL,Netherlands,vitesse,0.161616,9063453,AD013,2641,137,495,Netherlands,2024-01-01 21:00:00+00:00,view
4388390,NL,Netherlands,vitesse,0.161616,9063453,AD013,2641,137,495,Netherlands,2024-01-02 18:00:00+00:00,click
4388391,NL,Netherlands,vitesse,0.161616,9063453,AD013,2641,137,495,Netherlands,2024-01-02 19:00:00+00:00,view


In [35]:
trends_ads_perf_cus_merged = trends_ads_perf_cus_merged.drop(['country'], axis=1)
trends_ads_perf_cus_merged

,country_code,country_name,term,score,ads_criteria_id,ad_id,impressions,clicks,conversions,timestamp,user_action
0,CA,Canada,cyber monday deals,0.828283,2124,AD010,8722,1451,768,2024-01-01 09:00:00+00:00,view
1,CA,Canada,cyber monday deals,0.828283,2124,AD010,8722,1451,768,2024-01-01 18:00:00+00:00,view
2,CA,Canada,cyber monday deals,0.828283,2124,AD010,8722,1451,768,2024-01-02 06:00:00+00:00,click
3,CA,Canada,cyber monday deals,0.828283,1002806,AD010,8722,1451,768,2024-01-01 09:00:00+00:00,view
4,CA,Canada,cyber monday deals,0.828283,1002806,AD010,8722,1451,768,2024-01-01 18:00:00+00:00,view
...,...,...,...,...,...,...,...,...,...,...,...
4388388,NL,Netherlands,vitesse,0.161616,9063453,AD013,2641,137,495,2024-01-01 20:00:00+00:00,purchase
4388389,NL,Netherlands,vitesse,0.161616,9063453,AD013,2641,137,495,2024-01-01 21:00:00+00:00,view
4388390,NL,Netherlands,vitesse,0.161616,9063453,AD013,2641,137,495,2024-01-02 18:00:00+00:00,click
4388391,NL,Netherlands,vitesse,0.161616,9063453,AD013,2641,137,495,2024-01-02 19:00:00+00:00,view


In [36]:
trends_ads_perf_cus_merged = trends_ads_perf_cus_merged.drop_duplicates()

In [37]:
trends_ads_perf_cus_merged

,country_code,country_name,term,score,ads_criteria_id,ad_id,impressions,clicks,conversions,timestamp,user_action
0,CA,Canada,cyber monday deals,0.828283,2124,AD010,8722,1451,768,2024-01-01 09:00:00+00:00,view
1,CA,Canada,cyber monday deals,0.828283,2124,AD010,8722,1451,768,2024-01-01 18:00:00+00:00,view
2,CA,Canada,cyber monday deals,0.828283,2124,AD010,8722,1451,768,2024-01-02 06:00:00+00:00,click
3,CA,Canada,cyber monday deals,0.828283,1002806,AD010,8722,1451,768,2024-01-01 09:00:00+00:00,view
4,CA,Canada,cyber monday deals,0.828283,1002806,AD010,8722,1451,768,2024-01-01 18:00:00+00:00,view
...,...,...,...,...,...,...,...,...,...,...,...
4388388,NL,Netherlands,vitesse,0.161616,9063453,AD013,2641,137,495,2024-01-01 20:00:00+00:00,purchase
4388389,NL,Netherlands,vitesse,0.161616,9063453,AD013,2641,137,495,2024-01-01 21:00:00+00:00,view
4388390,NL,Netherlands,vitesse,0.161616,9063453,AD013,2641,137,495,2024-01-02 18:00:00+00:00,click
4388391,NL,Netherlands,vitesse,0.161616,9063453,AD013,2641,137,495,2024-01-02 19:00:00+00:00,view


In [38]:
"""import dask.dataframe as dd

# 將 Pandas 資料轉換為 Dask 資料
trends_ddf = dd.from_pandas(trends_ads_perf_cus_merged, npartitions=10)
google_ddf = dd.from_pandas(google_analytics_data, npartitions=10)

# 進行合併
merged_ddf = dd.merge(trends_ddf, google_ddf, left_on='country_name', right_on='country')
final_data = merged_ddf.compute()  # 將結果轉回 Pandas"""

"import dask.dataframe as dd\n\n# 將 Pandas 資料轉換為 Dask 資料\ntrends_ddf = dd.from_pandas(trends_ads_perf_cus_merged, npartitions=10)\ngoogle_ddf = dd.from_pandas(google_analytics_data, npartitions=10)\n\n# 進行合併\nmerged_ddf = dd.merge(trends_ddf, google_ddf, left_on='country_name', right_on='country')\nfinal_data = merged_ddf.compute()  # 將結果轉回 Pandas"

In [39]:
# 根據 country_name 或 country 拆分資料
unique_countries = trends_ads_perf_cus_merged['country_name'].unique()
merged_list = []

for country in unique_countries:
    temp_trends = trends_ads_perf_cus_merged[trends_ads_perf_cus_merged['country_name'] == country]
    temp_google_analytics = google_analytics_data[google_analytics_data['country'] == country]
    merged_chunk = pd.merge(temp_trends, temp_google_analytics, left_on='country_name', right_on='country')
    merged_list.append(merged_chunk)

# 合併所有分批的結果
final_data = pd.concat(merged_list, ignore_index=True)

In [40]:
final_data

,country_code,country_name,term,score,ads_criteria_id,ad_id,impressions,clicks,conversions,timestamp,user_action,country,pageviews,transactions
0,CA,Canada,cyber monday deals,0.828283,2124,AD010,8722,1451,768,2024-01-01 09:00:00+00:00,view,Canada,1,<NA>
1,CA,Canada,cyber monday deals,0.828283,2124,AD010,8722,1451,768,2024-01-01 09:00:00+00:00,view,Canada,1,<NA>
2,CA,Canada,cyber monday deals,0.828283,2124,AD010,8722,1451,768,2024-01-01 09:00:00+00:00,view,Canada,1,<NA>
3,CA,Canada,cyber monday deals,0.828283,2124,AD010,8722,1451,768,2024-01-01 09:00:00+00:00,view,Canada,1,<NA>
4,CA,Canada,cyber monday deals,0.828283,2124,AD010,8722,1451,768,2024-01-01 09:00:00+00:00,view,Canada,1,<NA>
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
173180715,NL,Netherlands,vitesse,0.161616,9063453,AD013,2641,137,495,2024-01-03 01:00:00+00:00,purchase,Netherlands,2,<NA>
173180716,NL,Netherlands,vitesse,0.161616,9063453,AD013,2641,137,495,2024-01-03 01:00:00+00:00,purchase,Netherlands,2,<NA>
173180717,NL,Netherlands,vitesse,0.161616,9063453,AD013,2641,137,495,2024-01-03 01:00:00+00:00,purchase,Netherlands,2,<NA>
173180718,NL,Netherlands,vitesse,0.161616,9063453,AD013,2641,137,495,2024-01-03 01:00:00+00:00,purchase,Netherlands,5,<NA>


In [41]:
chunk_size = 1000000  # 每次處理 100 萬筆資料
unique_chunks = []

for start in range(0, len(final_data), chunk_size):
    chunk = final_data.iloc[start:start + chunk_size]
    unique_chunk = chunk.drop_duplicates()
    unique_chunks.append(unique_chunk)

final_data = pd.concat(unique_chunks, ignore_index=True)
final_data

,country_code,country_name,term,score,ads_criteria_id,ad_id,impressions,clicks,conversions,timestamp,user_action,country,pageviews,transactions
0,CA,Canada,cyber monday deals,0.828283,2124,AD010,8722,1451,768,2024-01-01 09:00:00+00:00,view,Canada,1,<NA>
1,CA,Canada,cyber monday deals,0.828283,2124,AD010,8722,1451,768,2024-01-01 09:00:00+00:00,view,Canada,2,<NA>
2,CA,Canada,cyber monday deals,0.828283,2124,AD010,8722,1451,768,2024-01-01 09:00:00+00:00,view,Canada,3,<NA>
3,CA,Canada,cyber monday deals,0.828283,2124,AD010,8722,1451,768,2024-01-01 09:00:00+00:00,view,Canada,4,<NA>
4,CA,Canada,cyber monday deals,0.828283,2124,AD010,8722,1451,768,2024-01-01 09:00:00+00:00,view,Canada,5,<NA>
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29138890,NL,Netherlands,vitesse,0.161616,9063453,AD013,2641,137,495,2024-01-02 19:00:00+00:00,view,Netherlands,25,<NA>
29138891,NL,Netherlands,vitesse,0.161616,9063453,AD013,2641,137,495,2024-01-03 01:00:00+00:00,purchase,Netherlands,1,<NA>
29138892,NL,Netherlands,vitesse,0.161616,9063453,AD013,2641,137,495,2024-01-03 01:00:00+00:00,purchase,Netherlands,2,<NA>
29138893,NL,Netherlands,vitesse,0.161616,9063453,AD013,2641,137,495,2024-01-03 01:00:00+00:00,purchase,Netherlands,5,<NA>


In [42]:
# 上傳整合數據到 BigQuery
final_table_name = "looker-assignment-113356033.midterm_dataset.integrated_ad_analysis"
job = bigquery_client.load_table_from_dataframe(final_data, final_table_name)
job.result()  # 等待上傳完成
print(f"整合表格已成功上傳至 {final_table_name}")

整合表格已成功上傳至 looker-assignment-113356033.midterm_dataset.integrated_ad_analysis


**Question 2**

In [43]:
regional_trends = bigquery_client.query("""
WITH UniqueTerms AS (
  SELECT DISTINCT
    country_name,
    term,
    score
  FROM
    `looker-assignment-113356033.midterm_dataset.integrated_ad_analysis`
  WHERE
    score IS NOT NULL
),
RankedTerms AS (
  SELECT
    country_name,
    term,
    score,
    RANK() OVER (PARTITION BY country_name ORDER BY score DESC) AS rank
  FROM
    UniqueTerms
)
SELECT
  country_name,
  term,
  score,
  rank
FROM
  RankedTerms
WHERE
  rank <= 5
ORDER BY
  country_name,
  rank;

""").to_dataframe()

c:\Users\HungLin\anaconda3\Lib\site-packages\google\cloud\bigquery\table.py:1727: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


In [44]:
regional_trends

,country_name,term,score,rank
0,Brazil,jogo flamengo,1.000000,1
1,Brazil,ath. bilbao x real madrid,1.000000,1
2,Brazil,botafogo hoje,1.000000,1
3,Brazil,chapecoense x coritiba,1.000000,1
4,Brazil,cuiabá x bahia,1.000000,1
...,...,...,...,...
99,United Kingdom,lloyds share price,0.656566,3
100,United Kingdom,euromillions,0.606061,4
101,United Kingdom,ttrs,0.595960,5
102,United Kingdom,tottenham,0.595960,5


In [45]:
# 上傳整合數據到 BigQuery
regional_trends_name = "looker-assignment-113356033.midterm_dataset.region_trends"
job = bigquery_client.load_table_from_dataframe(regional_trends, regional_trends_name)
job.result()  # 等待上傳完成
print(f"地區排名表格已成功上傳至 {regional_trends_name}")

地區排名表格已成功上傳至 looker-assignment-113356033.midterm_dataset.region_trends


In [46]:
correlations = bigquery_client.query("""
SELECT
  country_name,
  CORR(score, conversions) AS corr_score_conversions,
  CORR(score, pageviews) AS corr_score_pageviews,
  CORR(score, transactions) AS corr_score_transactions,
  CORR(conversions, pageviews) AS corr_conversions_pageviews,
  CORR(conversions, transactions) AS corr_conversions_transactions,
  CORR(pageviews, transactions) AS corr_pageviews_transactions
FROM
  `looker-assignment-113356033.midterm_dataset.integrated_ad_analysis`
GROUP BY
  country_name;
""").to_dataframe()

c:\Users\HungLin\anaconda3\Lib\site-packages\google\cloud\bigquery\table.py:1727: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


In [47]:
correlations

,country_name,corr_score_conversions,corr_score_pageviews,corr_score_transactions,corr_conversions_pageviews,corr_conversions_transactions,corr_pageviews_transactions
0,Canada,NaN,-6.400761e-07,NaN,NaN,NaN,NaN
1,United Kingdom,NaN,1.667973e-06,NaN,NaN,NaN,NaN
2,Switzerland,NaN,-2.331277e-07,NaN,NaN,NaN,NaN
3,Japan,NaN,1.058318e-06,NaN,NaN,NaN,NaN
4,Brazil,NaN,1.670072e-06,NaN,NaN,NaN,NaN
5,Malaysia,NaN,3.782041e-07,NaN,NaN,NaN,NaN
6,Mexico,NaN,6.176652e-07,NaN,NaN,NaN,NaN
7,Ukraine,NaN,NaN,NaN,NaN,NaN,NaN
8,Italy,NaN,8.952265e-07,NaN,NaN,NaN,NaN
9,Spain,NaN,-3.987269e-06,NaN,NaN,NaN,NaN


In [48]:
# 上傳整合數據到 BigQuery
correlations_table_name = "looker-assignment-113356033.midterm_dataset.correlations"
job = bigquery_client.load_table_from_dataframe(correlations, correlations_table_name)
job.result()  # 等待上傳完成
print(f"相關係數表格已成功上傳至 {correlations_table_name}")

相關係數表格已成功上傳至 looker-assignment-113356033.midterm_dataset.correlations


In [49]:
ad_performance = bigquery_client.query("""
SELECT
  ad_id,
  country_name,
  SAFE_DIVIDE(SUM(clicks), SUM(impressions)) AS avg_ctr
FROM
  `looker-assignment-113356033.midterm_dataset.integrated_ad_analysis`
WHERE
  impressions > 0  -- 避免分母為 0
GROUP BY
  ad_id,
  country_name
ORDER BY
  ad_id;         -- 按廣告 ID 排序
""").to_dataframe()

c:\Users\HungLin\anaconda3\Lib\site-packages\google\cloud\bigquery\table.py:1727: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


In [50]:
ad_performance

,ad_id,country_name,avg_ctr
0,AD001,Norway,3.030445
1,AD002,United Kingdom,0.567224
2,AD003,Spain,0.360376
3,AD004,Switzerland,2.740538
4,AD005,Malaysia,1.444444
5,AD007,Indonesia,0.830469
6,AD008,Ukraine,1.464676
7,AD009,Turkey,0.572660
8,AD010,Canada,0.166361
9,AD011,Mexico,0.207663


In [51]:
# 上傳整合數據到 BigQuery
avg_ctr_table_name = "looker-assignment-113356033.midterm_dataset.avg_ctr"
job = bigquery_client.load_table_from_dataframe(ad_performance, avg_ctr_table_name)
job.result()  # 等待上傳完成
print(f"相關係數表格已成功上傳至 {avg_ctr_table_name}")

相關係數表格已成功上傳至 looker-assignment-113356033.midterm_dataset.avg_ctr


In [52]:
statistical_approximations = bigquery_client.query("""
SELECT
  APPROX_COUNT_DISTINCT(country_name) AS estimated_unique_countries
FROM
  `looker-assignment-113356033.midterm_dataset.integrated_ad_analysis`;
""").to_dataframe()

c:\Users\HungLin\anaconda3\Lib\site-packages\google\cloud\bigquery\table.py:1727: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


In [53]:
statistical_approximations

,estimated_unique_countries
0,17


In [54]:
# 上傳整合數據到 BigQuery
statistical_table_name = "looker-assignment-113356033.midterm_dataset.statistical"
job = bigquery_client.load_table_from_dataframe(statistical_approximations, statistical_table_name)
job.result()  # 等待上傳完成
print(f"相關係數表格已成功上傳至 {statistical_table_name}")

相關係數表格已成功上傳至 looker-assignment-113356033.midterm_dataset.statistical


In [70]:
create_ad_features_query = """
CREATE OR REPLACE TABLE `looker-assignment-113356033.midterm_dataset.ad_features` AS
SELECT DISTINCT
  ad_id,
  country_name,
  score,
  (clicks / NULLIF(impressions, 0)) AS ctr,
  user_action,
  conversions
FROM
  `looker-assignment-113356033.midterm_dataset.integrated_ad_analysis`
WHERE
  conversions IS NOT NULL; -- 移除缺失值
"""

In [66]:
create_model_query = """
CREATE OR REPLACE MODEL `looker-assignment-113356033.midterm_dataset.ad_conversion_model`
OPTIONS(
  model_type='LINEAR_REG',
  input_label_cols=['conversions'] -- 目標變量是 'conversions'
) AS
SELECT
  score,
  ctr,
  user_action,
  conversions
FROM
  `looker-assignment-113356033.midterm_dataset.ad_features`;
"""

In [67]:
evaluate_model_query = """
SELECT
  *
FROM
  ML.EVALUATE(
    MODEL `looker-assignment-113356033.midterm_dataset.ad_conversion_model`,
    (
      SELECT
        score,
        ctr,
        user_action,
        conversions
      FROM
        `looker-assignment-113356033.midterm_dataset.ad_features`
    )
  );
"""

In [68]:
def execute_query(query):
    query_job = bigquery_client.query(query)
    results = query_job.result()  # 等待查詢完成
    return results

In [71]:
# 執行每個查詢
print("Creating ad_features table...")
execute_query(create_ad_features_query)

print("Creating the linear regression model...")
execute_query(create_model_query)

print("Evaluating the model...")
evaluation_results = execute_query(evaluate_model_query)

# 顯示評估結果
for row in evaluation_results:
    print(dict(row))

Creating ad_features table...
Creating the linear regression model...
Evaluating the model...
{'mean_absolute_error': 252.72937533225098, 'mean_squared_error': 86166.5805328793, 'mean_squared_log_error': 1.0397661464566106, 'median_absolute_error': 271.00743398033865, 'r2_score': 0.10188801210947573, 'explained_variance': 0.10206455578608287}


In [77]:
create_prediction = """
CREATE OR REPLACE TABLE `looker-assignment-113356033.midterm_dataset.ad_predictions` AS
SELECT
  af.ad_id,
  af.country_name,
  AVG(af.conversions) AS avg_actual_conversions,
  AVG(predicted.predicted_conversions) AS avg_predicted_conversions
FROM
  `looker-assignment-113356033.midterm_dataset.ad_features` AS af
JOIN
  ML.PREDICT(
    MODEL `looker-assignment-113356033.midterm_dataset.ad_conversion_model`,
    (
      SELECT
        score,
        ctr,
        user_action,
        ad_id
      FROM
        `looker-assignment-113356033.midterm_dataset.ad_features`
    )
  ) AS predicted
ON
  af.ad_id = predicted.ad_id
GROUP BY
  af.ad_id,
  af.country_name;
"""

In [78]:
execute_query(create_prediction)